# Form the NB Regression Random Intercept Model: Simulated set


In [1]:
using DataFrames, Random, GLM, GLMCopula, Revise
using ForwardDiff, Test, LinearAlgebra
using LinearAlgebra: BlasReal, copytri!

Random.seed!(1235)

# sample size
N = 10000
# observations per subject
n = 5

variance_component_1 = 0.1
variance_component_2 = 0.9

r = 2
p = 0.5
μ = r * p * inv(1-p)
# true beta
β_true = log(μ)

dist = NegativeBinomial

Γ = variance_component_1 * ones(n, n) + variance_component_2 * Matrix(I, n, n)
vecd = [dist(r, p) for i in 1:n]
nonmixed_multivariate_dist = NonMixedMultivariateDistribution(vecd, Γ)

Y_Nsample = simulate_nobs_independent_vectors(nonmixed_multivariate_dist, N)

┌ Info: Precompiling GLMCopula [c47b6ae2-b804-4668-9957-eb588c99ffbc]
└ @ Base loading.jl:1278


10000-element Array{Array{Float64,1},1}:
 [2.0, 1.0, 7.0, 0.0, 7.0]
 [1.0, 6.0, 7.0, 2.0, 12.0]
 [3.0, 3.0, 0.0, 1.0, 0.0]
 [7.0, 4.0, 0.0, 0.0, 4.0]
 [2.0, 7.0, 4.0, 1.0, 2.0]
 [3.0, 0.0, 3.0, 1.0, 1.0]
 [10.0, 3.0, 0.0, 5.0, 2.0]
 [0.0, 2.0, 0.0, 7.0, 1.0]
 [0.0, 2.0, 8.0, 0.0, 1.0]
 [2.0, 3.0, 5.0, 1.0, 6.0]
 [1.0, 4.0, 4.0, 0.0, 0.0]
 [0.0, 1.0, 1.0, 2.0, 0.0]
 [2.0, 7.0, 1.0, 4.0, 0.0]
 ⋮
 [4.0, 2.0, 3.0, 2.0, 9.0]
 [1.0, 0.0, 1.0, 1.0, 0.0]
 [0.0, 1.0, 2.0, 5.0, 0.0]
 [4.0, 3.0, 2.0, 3.0, 2.0]
 [2.0, 3.0, 4.0, 1.0, 12.0]
 [0.0, 0.0, 5.0, 4.0, 7.0]
 [1.0, 7.0, 3.0, 3.0, 2.0]
 [4.0, 1.0, 4.0, 2.0, 2.0]
 [1.0, 6.0, 1.0, 1.0, 0.0]
 [0.0, 0.0, 2.0, 3.0, 1.0]
 [4.0, 2.0, 2.0, 0.0, 0.0]
 [0.0, 3.0, 0.0, 1.0, 1.0]

In [2]:
d = NegativeBinomial()
D = typeof(d)
gcs = Vector{GLMCopulaVCObs{Float64, D}}(undef, N)
for i in 1:N
    y = Float64.(Y_Nsample[i])
    X = ones(n, 1)
    V = [ones(n, n), Matrix(I, n, n)]
    gcs[i] = GLMCopulaVCObs(y, X, V, d)
end
gcm = GLMCopulaVCModel(gcs);

In [3]:
fill!(gcm.β, β_true)

1-element Array{Float64,1}:
 0.6931471805599453

Initialize β and σ2, here I just copy the solution for β and σ2 from MixedModels.jl over

In [4]:
fill!(gcm.Σ, 1.0)
update_Σ!(gcm)
@show gcm.Σ
# GLMCopula.loglikelihood!(gcm, true, true) this line doesnt work ArgumentError: NegativeBinomial: the condition zero(p) < p <= one(p) is not satisfied.

gcm.Σ = [10613.43343608239, 0.0]


2-element Array{Float64,1}:
 10613.43343608239
     0.0

Closer Look at Observation 1
i =1

In [5]:
gc = gcm.data[1]
β  = gcm.β
Σ  = gcm.Σ
τ  = gcm.τ

@show β
@show Σ
@show τ

n_i  = length(gc.y)

β = [0.6931471805599453]
Σ = [10613.43343608239, 0.0]
τ = [0.05314691797819666]


5

## Loglikelihood for observation i = 1, j in [1, n_1]
$$\mathcal{L}(\mathbf{\beta})_1 =  - \ln \Big[1\! +\! \frac{1}{2}tr(\mathbf{\Gamma_{1}})\Big] +
\ln \Big\{1\!+\!\frac{1}{2}\mathbf{r_1}(\mathbf{\beta})^t \mathbf{\Gamma_1} \mathbf{r_1}(\mathbf{\beta})\Big\} +  \sum_{j=1}^{n_1}y_{1j}log(\mu_{1j}(\mathbf{\beta})) - \mu_{1j}(\mathbf{\beta})$$

### First I want to check if the mean and residuals are updated and standardized at this point

In [6]:
@show gc.res
update_res!(gcm)
@show gc.res
standardize_res!(gcm)
@show gc.res

@test gc.η == gc.X*β                         # systematic linear component
@test gc.μ == exp.(gc.η)                     # mu = ginverse of XB = mean component for GLM
@test gc.varμ == gc.μ .* (1 .+ gc.μ ./gc.d.r)                # variance of the GLM response as a function of mean mu
@test gc.res == (gc.y .- gc.μ) ./ sqrt.(gc.varμ) # standardized residual for GLM outcome

gc.res = [2.82842712474619, 2.1213203435596424, 6.363961030678928, 1.414213562373095, 6.363961030678928]
gc.res = [0.0, -1.0, 5.0, -2.0, 5.0]
gc.res = [0.0, -0.4082482904638631, 2.041241452319315, -0.8164965809277261, 2.041241452319315]


Test Passed

### Next I want to check if the hard coded terms in the loglikelihood are correct

$$\text{Term 1 }= - \ln \Big[1\! +\! \frac{1}{2}tr(\mathbf{\Gamma_{1}})\Big]$$

In [7]:
Γ_est = Σ[1] * gc.V[1] + Σ[2] * gc.V[2]
trace_gamma = tr(Γ_est)

term1 = -log(1 + 0.5 * trace_gamma)
@show term1;

term1 = -10.186204202288135


In [8]:
tsum = dot(Σ, gc.t)
@show -log(1 + tsum);

-(log(1 + tsum)) = -10.186204202288135


In [9]:
@show gc.res
m = length(gc.V)
for k in 1:m
    mul!(gc.storage_n, gc.V[k], gc.res) 
    gc.q[k] = dot(gc.res, gc.storage_n) / 2 
end
qsum  = dot(Σ, gc.q) 
inv1pq = inv(1 + qsum)

@show  log(1 + qsum)

gc.res = [0.0, -0.4082482904638631, 2.041241452319315, -0.8164965809277261, 2.041241452319315]
log(1 + qsum) = 10.676812505432654


10.676812505432654

$$\text{Term 2} = \ln \Big\{1\!+\!\frac{1}{2}\mathbf{r_1}(\mathbf{\beta})^t \mathbf{\Gamma_1} \mathbf{r_1}(\mathbf{\beta})\Big\}$$

In [10]:
# term 2:
quad_form_standardized_res = transpose(gc.res) * Γ_est * gc.res
@show quad_form_standardized_res

term2 = log(1 + 0.5 * quad_form_standardized_res) 
@show term2

quad_form_standardized_res = 86676.37306133953
term2 = 10.676812505432654


10.676812505432654

### In the loglikelihood function I have:

$$\text{Term1 + Term2} =  - \ln \Big[1\! +\! \frac{1}{2}tr(\mathbf{\Gamma_{1}})\Big] +
\ln \Big\{1\!+\!\frac{1}{2}\mathbf{r_1}(\mathbf{\beta})^t \mathbf{\Gamma_1} \mathbf{r_1}(\mathbf{\beta})\Big\}$$

In [11]:
logl_hard_coded_obs1 = term1 + term2
copula_logl_function = GLMCopula.copula_loglikelihood_addendum(gc, Σ)
@show gc.res
@show logl_hard_coded_obs1
@show copula_logl_function
@test copula_logl_function ≈ logl_hard_coded_obs1

gc.res = [0.0, -0.4082482904638631, 2.041241452319315, -0.8164965809277261, 2.041241452319315]
logl_hard_coded_obs1 = 0.4906083031445192
copula_logl_function = 0.4906083031445192


Test Passed

### Part of Loglikelihood that comes from the Density using GLM.jl

$$\text{Term 3} = \sum_{j=1}^{n_1} NB(y_{ij})$$

In [12]:
term3 = 0.0
for j in 1:n_i
   term3 += GLM.logpdf(NegativeBinomial(gc.d.r, gc.d.r/(gc.μ[j]+gc.d.r)), gc.y[j]) 
end
term3

-12.38596828117934

In [13]:
logl_component_nb = 0.0
logl_component_nb += component_loglikelihood(gc, τ[1], logl_component_nb)

-12.38596828117934

$$\mathcal{L}(\mathbf{\beta})_1 =  - \ln \Big[1\! +\! \frac{1}{2}tr(\mathbf{\Gamma_{1}})\Big] +
\ln \Big\{1\!+\!\frac{1}{2}\mathbf{r_1}(\mathbf{\beta})^t \mathbf{\Gamma_1} \mathbf{r_1}(\mathbf{\beta})\Big\} +  \sum_{j=1}^{n_1} NB(y_{1j})$$

In [14]:
logl_hard = term1 + term2 + term3

-11.89535997803482

In [15]:
logl_my_function = copula_loglikelihood(gc, β, τ[1], Σ)

-11.89535997803482

In [16]:
@test logl_hard ≈ logl_my_function

Test Passed

# A Closer Look at the Gradient for observation i=1

$$\begin{eqnarray*}
\nabla_\beta &=& \sum_{i=1}^n \sum_j \nabla \ln f_{ij}(y_{ij} \mid \mathbf{\beta}) + \sum_{i=1}^n
\frac{\nabla \mathbf{r_i(\mathbf{\beta})}\mathbf{\Gamma_i}\mathbf{r_i(\mathbf{\beta})}}{1+\frac{1}{2}\mathbf{r_i}(\mathbf{\beta})^t \mathbf{\Gamma_i} \mathbf{r_i(\mathbf{\beta})}}
\end{eqnarray*}
$$

The gradient is made of two terms. The first is from the GLM component loglikelihood that corresponds to the Logistic Regression density. The second part is specific to our copula model. We start with Term 1 for observation 1:

$$\begin{eqnarray*}
    \text{Term 1} &=& \sum_{j=1}^{n_1} \frac{(y_{1j}-\mu_{1j}) \mu_{1j}'(\eta_{1j})}{\sigma_{1j}^2} \mathbf{x}_{1j}
\end{eqnarray*}
$$

We will check if the field $\mu_{1j}'$ or `mueta` from the GLM.jl package matches our theoretical value

In [17]:
@test gc.dμ == exp.(gc.η)                  # derivative of mean with respect to systematic component

Test Passed

In [18]:
function nb_gradient(y, X, dμ, σ2, μ)
    grad = zeros(size(X, 2))
    for j in 1:length(y)
        grad += (y[j] - μ[j]) * dμ[j]/σ2[j] * X[j, :]
    end
    grad
end

# check if glm gradient is right
term1_gradient = nb_gradient(gc.y, gc.X, gc.dμ, gc.varμ, gc.μ)

1-element Array{Float64,1}:
 2.3333333333333335

In [19]:
term1_grad_fctn = GLMCopula.glm_gradient(gc, β, τ)
@test term1_gradient == term1_grad_fctn

Test Passed

### Copula density specific gradient portion

$$ \text{Term 2} = \frac{\nabla \mathbf{r_1(\mathbf{\beta})}^\top\mathbf{\Gamma_1}\mathbf{r_1(\mathbf{\beta})}}{1+\frac{1}{2}\mathbf{r_1}(\mathbf{\beta})^t \mathbf{\Gamma_i} \mathbf{r_1(\mathbf{\beta})}}
$$

Notice the second term uses a critical value that will come up in the Hessian as well. For the first observation, $i = 1,$ we have $n_1 = 5$ and $p = 1$ in the simulated dataset. $\nabla \mathbf{r_1(\mathbf{\beta})}$ which is an $5 \times 1$ matrix of differentials. 

Each column of $\nabla \mathbf{r_1(\mathbf{\beta})}^\top$ is a $p \times 1$ vector $\nabla r_{1j}(\mathbf{\beta})$

$$
\begin{eqnarray*}
\nabla r_{1j}(\mathbf{\beta}) &=&  -\frac{1}{\sigma_{1j}(\mathbf{\beta})} \nabla \mu_{1j}(\mathbf{\beta})- \frac{1}{2} \frac{y_{1j}-\mu_{1j}(\mathbf{\beta})}{\sigma_{1j}^3(\mathbf{\beta})} \nabla \sigma_{1j}^2(\mathbf{\beta})\\
&=&  -\frac{1}{\sigma_{1j}(\mathbf{\beta})} \nabla \mu_{1j}(\mathbf{\beta})- \frac{1}{2\sigma_{1j}^2(\mathbf{\beta})}r_{1j}(\mathbf{\beta}) \nabla \sigma_{1j}^2(\mathbf{\beta})
\end{eqnarray*}
$$


where 

$$
\begin{eqnarray*}
\nabla \mu_{1j}(\mathbf{\beta}) &=& e^{\eta_{1j}(\mathbf{\beta})} * \mathbf{x_{1j}}
\end{eqnarray*}
$$

In [20]:
# for j = 1 and j = 2, ..., j = end; lets take a look at the first two columns 
∇μβ1 = exp.(gc.η[1]) .* gc.X[1, :]
∇μβ2 = exp.(gc.η[2]) .* gc.X[2, :]
# ...
∇μβend = exp.(gc.η[end]) .* gc.X[end, :]

@show ∇μβ1
@show ∇μβ2
@show ∇μβend

∇μβ1 = [2.0]
∇μβ2 = [2.0]
∇μβend = [2.0]


1-element Array{Float64,1}:
 2.0

and  
$$  \begin{eqnarray*}
    \nabla \sigma_{ij}^2(\beta) = \frac{d\sigma_{ij}^2(\beta)}{d\mu_{ij}(\beta)} \frac{d\mu_{ij}(\beta)}{d\eta_{ij}(\beta)} \frac{d\eta_{ij}(\beta)}{d\beta} = (1 + \frac{\mu_{ij}}{r}) + \frac{\mu_{ij}}{r} * e^{\eta_{ij}(\beta)} \mathbf{x}_{ij} 
\end{eqnarray*}$$

In [69]:
# for j = 1 and j = 2 ,... , j = end; lets take a look at the first two columns 
∇σ2β1 = ((1 + gc.μ[1] * inv(gc.d.r)) .+ gc.dμ[1] * inv(gc.d.r)) * gc.μ[1] .* gc.X[1, :]
∇σ2β2 = ((1 + gc.μ[2] * inv(gc.d.r)) .+ gc.dμ[2] * inv(gc.d.r)) * gc.μ[2] .* gc.X[2, :]
# ...
∇σ2βend = ((1 + gc.μ[end] * inv(gc.d.r)) .+ gc.dμ[end] * inv(gc.d.r)) * gc.μ[end] .* gc.X[end, :]

@show ∇σ2β1
@show ∇σ2β2
@show ∇σ2βend


∇σ2β1 = [10.0]
∇σ2β2 = [10.0]
∇σ2βend = [10.0]


1-element Array{Float64,1}:
 10.0

$$
\begin{eqnarray*}
\nabla r_{1j}(\mathbf{\beta}) &=&  -\frac{1}{\sigma_{1j}(\mathbf{\beta})} \nabla \mu_{1j}(\mathbf{\beta})- \frac{1}{2} \frac{y_{1j}-\mu_{1j}(\mathbf{\beta})}{\sigma_{1j}^3(\mathbf{\beta})} \nabla \sigma_{1j}^2(\mathbf{\beta})\\
&=&  -\frac{1}{\sigma_{1j}(\mathbf{\beta})} \nabla \mu_{1j}(\mathbf{\beta})- \frac{1}{2\sigma_{1j}^2(\mathbf{\beta})}r_{1j}(\mathbf{\beta}) \nabla \sigma_{1j}^2(\mathbf{\beta})
\end{eqnarray*}
$$


In [86]:
-1/(sqrt(gc.varμ[2])) * ∇μβ2[1] - (0.5 * inv(gc.varμ[2])) * gc.res[2] * ∇σ2β2[1]

-0.4762896722078403

In [95]:
∇resβ1 = -1/(sqrt(gc.varμ[1])) * ∇μβ1[1] - (0.5 * inv(gc.varμ[1])) * gc.res[1] * ∇σ2β1[1]
∇resβ2 = -1/(sqrt(gc.varμ[2])) * ∇μβ2[1] - (0.5 * inv(gc.varμ[2])) * gc.res[2] * ∇σ2β2[1]
# ...
∇resβend = -1/(sqrt(gc.varμ[end])) * ∇μβend[1] - (0.5 * inv(gc.varμ[end])) * gc.res[end] * ∇σ2βend[1]

@show ∇resβ1
@show ∇resβ2
@show ∇resβend

∇resβ1 = -0.8164965809277261
∇resβ2 = -0.4762896722078403
∇resβend = -2.517531124527155


-2.517531124527155

In [100]:
std_res_differential!(gc)
@test gc.∇resβ[1, :] == [∇resβ1]

Test Passed

In [101]:
@test gc.∇resβ[2, :] == [∇resβ2]

Test Passed

In [102]:
@test gc.∇resβ[end, :] == [∇resβend]

Test Passed

### Gradient portion of Copula specific model

$$\begin{eqnarray*}
\text{Term 2} &=& \sum_{i=1}^n
\frac{\nabla \mathbf{r_i(\mathbf{\beta})}\mathbf{\Gamma_i}\mathbf{r_i(\mathbf{\beta})}}{1+\frac{1}{2}\mathbf{r_i}(\mathbf{\beta})^t \mathbf{\Gamma_i} \mathbf{r_i(\mathbf{\beta})}}
\end{eqnarray*}
$$

Again for observation $ i = 1$ we have:  

$$ \text{Term 2} = \frac{\nabla \mathbf{r_1(\mathbf{\beta})}^\top\mathbf{\Gamma_1}\mathbf{r_1(\mathbf{\beta})}}{1+\frac{1}{2}\mathbf{r_1}(\mathbf{\beta})^t \mathbf{\Gamma_i} \mathbf{r_1(\mathbf{\beta})}}
$$

## To do: Fix copula_gradient_addendum

In [103]:
Γ_est = Σ[1] * gc.V[1] + Σ[2] * gc.V[2]

grad_t2_numerator = transpose(gc.∇resβ) * Γ_est * gc.res       # new term ∇resβ^t * Γ * res
@show grad_t2_numerator

quadratic_form = transpose(gc.res) * Γ_est * gc.res
@show quadratic_form 
@test quadratic_form ≈ quad_form_standardized_res # from the loglikelihood 'qsum'

grad_t2_denominator = inv(1 + 0.5 * quadratic_form)
@show grad_t2_denominator

gradient_term2 = grad_t2_numerator * grad_t2_denominator

grad_t2_numerator = [-196053.7009720775]
quadratic_form = 86676.37306133953
grad_t2_denominator = 2.3073806410564068e-5


1-element Array{Float64,1}:
 -4.523705142304332

In [104]:
gradient_term2_function = GLMCopula.copula_gradient_addendum(gc, β, τ[1], Σ)

1-element Array{Float64,1}:
 -11.259253438412616

In [107]:
@test gradient_term2 ≈ gradient_term2_function

Test Failed at In[107]:1
  Expression: gradient_term2 ≈ gradient_term2_function
   Evaluated: [-4.523705142304332] ≈ [-11.259253438412616]


LoadError: [91mThere was an error during testing[39m

In [30]:
gradient_hard_code = term1_gradient + gradient_term2

1-element Array{Float64,1}:
 -60.5223591702637

In [31]:
full_gradient_my_function = copula_gradient(gc, β, τ, Σ)

1-element Array{Float64,1}:
 -8.925920105079282

In [32]:
@test full_gradient_my_function ≈ gradient_hard_code

Test Failed at In[32]:1
  Expression: full_gradient_my_function ≈ gradient_hard_code
   Evaluated: [-8.925920105079282] ≈ [-60.5223591702637]


LoadError: [91mThere was an error during testing[39m

## Let's now use the ForwardDiff.jl package to check if our matrix calculus is correct.

I want to start by checking my calculation of the gradient. 

    (1) I will modify the functions that reflect parts of the loglikelihood above in section 1 to use the package properly. 
    (2) I will then compare the results to that from my gradient functions above in section 2 


###  Test if the gradient of the component loglikelihood matches our gradient function

In [ ]:
function poisson_density(β::Vector)
    η = gc.X*β                        # systematic linear component
    μ = exp.(η)   # mu = ginverse of XB = mean component for GLM = [p]
    dμ = exp.(η)
    varμ = dμ
    logl = sum(gc.y .* log.(μ) .- μ)
end

logl_term3 = poisson_density(β)
@show logl_term3

g = x -> ForwardDiff.gradient(poisson_density, x)

gradientmagictest = g(β)
@show gradientmagictest
@show term1_grad_fctn
@test term1_grad_fctn == gradientmagictest

# Now we check the gradient of the matrix of differentials residual vector

Lets start with $i = 1, j = 1$ so the first observation of the $i^{th}$ group has the following gradient of the standardized residual.

In [64]:
std_res_differential!(gc)
gc.∇resβ

5×1 Array{Float64,2}:
  -0.8164965809277261
  11.430952132988166
 -62.053740150507174
  23.678400846904058
 -62.053740150507174

In [49]:
function standardized_residual_firstobs(β::Vector)
    η = gc.X*β                        # systematic linear component
    μ = exp.(η) # mu = ginverse of XB = mean component for GLM = [p]
    varμ = μ .* (1 .+ μ ./gc.d.r)
    res = (gc.y[1] - μ[1]) / sqrt(varμ[1])
end

g2 = x -> ForwardDiff.gradient(standardized_residual_firstobs, x)
gradientmagictest2 = g2(β)
@show gradientmagictest2

# @test gc.∇resβ[1, :] == gradientmagictest2

length(varμ) = 5
gradientmagictest2 = [-0.8164965809277261]


1-element Array{Float64,1}:
 -0.8164965809277261

In [79]:
standardized_residual_secondobs(β)

-0.4082482904638631

In [81]:
function standardized_residual_secondobs(β::Vector)
    η = gc.X*β                        # systematic linear component
    μ = exp.(η) # mu = ginverse of XB = mean component for GLM = [p]
    varμ = μ .* (1 .+ μ ./gc.d.r)
    res = (gc.y[2] - μ[2]) / sqrt(varμ[2])
    res
end

g_second = x -> ForwardDiff.gradient(standardized_residual_secondobs, x)
gradientmagictest22 = g_second(β)
@show gradientmagictest22

gradientmagictest22 = [-0.47628967220784024]


1-element Array{Float64,1}:
 -0.47628967220784024

In [62]:
function standardized_residual_lastobs(β::Vector)
    η = gc.X*β                        # systematic linear component
    μ = exp.(η) # mu = ginverse of XB = mean component for GLM = [p]
    varμ = μ .* (1 .+ μ ./gc.d.r)
    res = (gc.y[5] - μ[5]) / sqrt(varμ[5])
    res
end

g_last = x -> ForwardDiff.gradient(standardized_residual_lastobs, x)
gradientmagictest_5 = g_last(β)
@show gradientmagictest_5

gradientmagictest_5 = [-2.5175311245271557]


1-element Array{Float64,1}:
 -2.5175311245271557

## Now I will check the part of the Loglikelihood that is specific to our density

In [ ]:
function copula_loglikelihood_addendum1(β::Vector)
  m = length(gc.V)
  η = gc.X*β                        # systematic linear component
  μ = exp.(η) # mu = ginverse of XB = mean component for GLM = [p]
  varμ = exp.(η)
  res = (gc.y .- μ) ./ sqrt.(varμ)
  trace_gamma = tr(Γ_est)

  term1 = -log(1 + 0.5 * trace_gamma)
  quad_form_standardized_res = transpose(res)* Γ_est * res
  term2 = log(1 + 0.5 * quad_form_standardized_res)
  logl_hard_coded_obs1 = term1 + term2
  logl_hard_coded_obs1
end

g3 = x -> ForwardDiff.gradient(copula_loglikelihood_addendum1, x)

@show copula_loglikelihood_addendum1(β)

gradientmagictest3 = g3(β)
@show gradientmagictest3
@show gradient_term2_function
@test gradient_term2_function ≈ gradientmagictest3


## Now I will put together both the parts of the loglikelihood and both the parts of the gradient to check alltogether now

In [ ]:
function full_loglikelihood(β::Vector)
    logl = 0.0
    logl = poisson_density(β) + copula_loglikelihood_addendum1(β)
    logl
end

@show full_loglikelihood(β)

g4 = x -> ForwardDiff.gradient(full_loglikelihood, x)

gradientmagictest4 = g4(β)
@show gradientmagictest4

full_gradient_function = copula_gradient(gc, β, τ, Σ)
@show full_gradient_function
@test full_gradient_function ≈ gradientmagictest4